<a href="https://colab.research.google.com/github/BrunoStrufaldi/Automa-o-GoogleColab/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import re

# ==============================================================================
# Funções de limpeza
# ==============================================================================
def remover_emojis(texto):
    if not texto: return ""
    emoji_pattern = re.compile("["
        "\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF" "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF" "\U00002702-\U000027B0" "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', texto).strip()

def remover_caracteres_controle(texto):
    if not texto: return ""
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', texto)

# ==============================================================================
# NOMES DOS ARQUIVOS
# ==============================================================================
NOME_DO_ARQUIVO_ENTRADA = '/projeto pyton (1).xlsx'
NOME_DO_ARQUIVO_SAIDA = 'descricoes_finais.xlsx'
# ==============================================================================

try:
    df = pd.read_excel(NOME_DO_ARQUIVO_ENTRADA)
    print(f"--- Arquivo Excel '{NOME_DO_ARQUIVO_ENTRADA}' lido com sucesso! Processando os produtos... ---")

    lista_de_descricoes = []

    def formatar_lista_com_emoji(lista_de_itens, emoji="📌"):
        itens_validos = [item.strip() for item in lista_de_itens if item.strip()]
        return "\n".join([f"  {emoji} {item}" for item in itens_validos])

    for index, row in df.iterrows():
        texto_bruto_da_planilha = str(row.get('descrição', '')).strip()
        texto_sem_emojis = remover_emojis(texto_bruto_da_planilha)
        descricao_bruta = remover_caracteres_controle(texto_sem_emojis)

        if not descricao_bruta: continue

        linhas = [linha.strip() for linha in descricao_bruta.split('\n') if linha.strip()]

        if not linhas: continue

        primeira_linha_bruta = linhas[0].strip('*')

        # --- AJUSTE: LÓGICA DE TÍTULO APRIMORADA (SINGULAR/PLURAL) ---
        PALAVRAS_CHAVE_STEMS = ["espalhador", "grelha", "queimador"]
        posicao_corte = len(primeira_linha_bruta)

        for stem in PALAVRAS_CHAVE_STEMS:
            # Encontra todas as ocorrências do radical da palavra (ex: "queimador" em "queimadores")
            matches = list(re.finditer(stem, primeira_linha_bruta.lower()))

            # Se houver mais de uma ocorrência, significa que a palavra se repetiu
            if len(matches) > 1:
                # Pega a posição de início da segunda ocorrência
                segunda_ocorrencia_pos = matches[1].start()

                # Atualiza a posição de corte se esta for a repetição mais próxima do início
                if segunda_ocorrencia_pos < posicao_corte:
                    posicao_corte = segunda_ocorrencia_pos

        primeira_linha = primeira_linha_bruta[:posicao_corte].strip()
        titulo_anuncio = primeira_linha.upper()
        # --- FIM DO AJUSTE ---

        compatibilidade_items = [
            re.sub(r'.*COMPAT[ÍI]VEIS COM OS MODELOS\s*[:-]*\s*', '', linha, flags=re.IGNORECASE)
            for linha in linhas if "compatív" in linha.lower()
        ]
        conteudo_items = [
            linha.lstrip('- ') for linha in linhas
            if re.match(r'^\s*(\d+\s*-|-\s*)', linha)
        ]

        material_items = []
        material_encontrado = False
        PALAVRAS_DE_PARADA_MATERIAL = ["prazo de validade", "atenção", "benefícios", "dúvidas"]
        for linha in linhas:
            linha_lower = linha.lower()
            if "material:" in linha_lower or "materiais:" in linha_lower:
                material_encontrado = True
                try:
                    material_info = linha.split(':', 1)[1].strip()
                    if material_info:
                        material_items.append(material_info)
                except IndexError:
                    pass
                continue

            if material_encontrado:
                 if any(palavra in linha_lower for palavra in PALAVRAS_DE_PARADA_MATERIAL):
                     material_encontrado = False
                     continue

                 if linha.strip():
                     material_items.append(linha.strip())

        compatibilidade_formatado = ""
        if compatibilidade_items:
            itens_compat_str = formatar_lista_com_emoji(compatibilidade_items, emoji="📌")
            compatibilidade_formatado = f"\n\n🔗 **COMPATIBILIDADE:**\n{itens_compat_str}"

        conteudo_formatado = formatar_lista_com_emoji(conteudo_items)
        conteudo_kit_raw = " ".join(conteudo_items)
        conteudo_lower_completo = (titulo_anuncio + conteudo_kit_raw).lower()

        if not material_items:
            inferred_materials = []
            if "queimador" in conteudo_lower_completo: inferred_materials.append("Queimadores: Alumínio")
            if "espalhador" in conteudo_lower_completo: inferred_materials.append("Espalhadores: Aço Esmaltado")
            if "grelha" in conteudo_lower_completo: inferred_materials.append("Grelhas: Arame De Aço Esmaltado")
            material_formatado = formatar_lista_com_emoji(inferred_materials)
        else:
            material_formatado = formatar_lista_com_emoji(material_items)

        beneficios = ["Chamas mais estáveis e aquecimento uniforme", "Alta durabilidade", "Fácil instalação", "Reposição perfeita"]
        if "SÓ GRELHA" in conteudo_kit_raw.upper() or ("GRELHA" in conteudo_kit_raw.upper() and len(conteudo_items) <= 2):
            beneficios.remove("Chamas mais estáveis e aquecimento uniforme")
        beneficios_formatado = "".join([f"📌 {b}\n" for b in beneficios])

        TODOS_COMPONENTES = {"espalhadores", "queimadores", "grelhas"}
        componentes_presentes = set()
        if "espalhador" in conteudo_lower_completo: componentes_presentes.add("espalhadores")
        if "queimador" in conteudo_lower_completo: componentes_presentes.add("queimadores")
        if "grelha" in conteudo_lower_completo: componentes_presentes.add("grelhas")

        componentes_ausentes = sorted(list(TODOS_COMPONENTES - componentes_presentes))

        obs_final = ""
        if len(componentes_ausentes) == 2:
            obs_final = f"Obs: este anúncio não inclui {componentes_ausentes[0]} e {componentes_ausentes[1]}."
        elif len(componentes_ausentes) == 1:
            obs_final = f"Obs: este anúncio não inclui {componentes_ausentes[0]}."
        elif len(componentes_presentes) > 0 and len(componentes_ausentes) == 0:
             obs_final = "Obs: anúncio referente a um kit completo com queimadores, espalhadores e grelhas."
        else:
            if componentes_presentes:
                itens_str = " e ".join(sorted(list(componentes_presentes)))
                artigo = "aos"
                if "grelhas" in itens_str and len(componentes_presentes) == 1:
                    artigo = "às"
                obs_final = f"Obs: este anúncio se refere apenas {artigo} {itens_str}."

        descricao_final = f"""🔥 **{titulo_anuncio}** 🔥

🔹 Renove seu fogão com peças originais de **alta qualidade, encaixe perfeito e longa durabilidade!**
✅ Resistência e acabamento premium garantidos.{compatibilidade_formatado}

📦 **CONTEÚDO DO KIT:**
{conteudo_formatado}

⚙️ **MATERIAL:**
{material_formatado}

🕒 **PRAZO DE VALIDADE:**
📌 Indeterminado

💪 **BENEFÍCIOS:**
{beneficios_formatado}
⚠️ **ATENÇÃO:**
📌 Antes da compra, compare as fotos, medidas e quantidades de peças do anúncio com as peças do seu fogão para garantir o encaixe correto.
📌 {obs_final}

📩 **DÚVIDAS?**
📌 Estamos à disposição para ajudar. Entre em contato pelo campo de mensagens que teremos prazer em responder.

🔥 Garanta já o seu kit e deixe seu fogão como novo!
"""

        lista_de_descricoes.append(descricao_final)

    print("\n--- Criando a planilha Excel com os resultados... ---")
    df_saida = pd.DataFrame(lista_de_descricoes, columns=['Descrições Geradas'])
    df_saida.to_excel(NOME_DO_ARQUIVO_SAIDA, index=False)

    print(f"\n--- FIM: Todas as descrições foram geradas e salvas em '{NOME_DO_ARQUIVO_SAIDA}'. ---")
    print("--- Você pode baixar a planilha na aba 'Arquivos' à esquerda. ---")

except FileNotFoundError:
    print(f"\n### ERRO: Arquivo '{NOME_DO_ARQUIVO_ENTRADA}' não encontrado. Verifique o nome da coluna e do arquivo e se fez o upload. ###")
except Exception as e:
    print(f"\n### ERRO INESPERADO: {e} ###")

--- Arquivo Excel '/projeto pyton (1).xlsx' lido com sucesso! Processando os produtos... ---

--- Criando a planilha Excel com os resultados... ---

--- FIM: Todas as descrições foram geradas e salvas em 'descricoes_finais.xlsx'. ---
--- Você pode baixar a planilha na aba 'Arquivos' à esquerda. ---
